# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

Do not modify or remove any of the code in these cells.

In [1]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.metrics.distance  import edit_distance

from imblearn.metrics import classification_report_imbalanced

from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import spellchecker
from spellchecker import SpellChecker
import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lizet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lizet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\lizet\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') # no labels

train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [4]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


## Overview

Refer to this diagram to guide you while you build your model. Some of the steps will be fleshed out in this template.

![Overview](process_overview_final.jpg)

## Basic preprocessing

Here is a template you may use for your initial base model. 

### Removing URL's
Write a function that removes URL's from a single tweet. 

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet with URL's removed as a single object

In [5]:
# define function that removes URL from single tweet
def remove_urls(text):
    pattern_url = re.compile(r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+')
    subs_url = r' '
    return pattern_url.sub(subs_url, text)


**Original object:**

In [6]:
tweet = train['message'][2]
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [7]:
remove_urls(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late    …'

In [8]:

train.head()


,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


**Expected output:**
```python
remove_urls(tweet)

>>'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '
```

In [9]:
def remove_punc(text):
    return ''.join([l for l in text if l not in string.punctuation])

In [10]:
remove_punc(tweet)

'RT RawStory Researchers say we have three years to act on climate change before it’s too late httpstcoWdT0KdUr2f httpstcoZ0ANPT…'

In [11]:
spell = SpellChecker()

def Correct_spelling(text):
    misspelled = spell.unknown(text)
    return spell.correction(text)


In [12]:
text = 'speling'

In [13]:
Correct_spelling(text)

'spelling'

### Convert to lowercase

Write a function that converts a single tweet to lowercase.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in lowercase as a single object

In [14]:
def to_lower(text):
    return text.lower()

**Original object:**

In [15]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [16]:
to_lower(tweet)

'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

In [17]:
import re
def remove_handle(text):
    handle = re.compile('@[^\s]+')
    subs_handle = r' '
    return handle.sub(subs_handle,text)

In [18]:
remove_handle(tweet)

'RT   Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [19]:
def stop_words(text):
    return [t for t in text if t not in stopwords.words('english')]

In [20]:
stop_words(tweet)

['R',
 'T',
 ' ',
 '@',
 'R',
 'w',
 'S',
 'r',
 ':',
 ' ',
 'R',
 'e',
 'e',
 'r',
 'c',
 'h',
 'e',
 'r',
 ' ',
 ' ',
 'w',
 'e',
 ' ',
 'h',
 'v',
 'e',
 ' ',
 'h',
 'r',
 'e',
 'e',
 ' ',
 'e',
 'r',
 ' ',
 ' ',
 'c',
 ' ',
 'n',
 ' ',
 'c',
 'l',
 'e',
 ' ',
 'c',
 'h',
 'n',
 'g',
 'e',
 ' ',
 'b',
 'e',
 'f',
 'r',
 'e',
 ' ',
 '’',
 ' ',
 ' ',
 'l',
 'e',
 ' ',
 'h',
 'p',
 ':',
 '/',
 '/',
 '.',
 'c',
 '/',
 'W',
 'T',
 '0',
 'K',
 'U',
 'r',
 '2',
 'f',
 ' ',
 'h',
 'p',
 ':',
 '/',
 '/',
 '.',
 'c',
 '/',
 'Z',
 '0',
 'A',
 'N',
 'P',
 'T',
 '…']

**Expected output:**

```python 
to_lower(tweet)
>> 'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

### Stemming or lemmatising

Write a function that derives the root words for each of the words in a tweet. You may use stemming _or_ lemmatising here.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in root words as a _single_ object

In [21]:
stemmer = SnowballStemmer('english')
def get_stems(text):
    return " ".join([stemmer.stem(w) for w in str(text).split()])


**Original object:**

In [22]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [23]:
get_stems(tweet)

'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**
```python
get_roots(tweet)
```
Using `SnowballStemmer()`
```python
>> 'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

Using `WordNetLemmatizer()`
```python
>> 'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'
```

### Other preprocessing techniques 

Come back here once you have submitted your base model to add any other preprocessing functions yuo wish to add

In [24]:
# YOUR CODE HERE
lemmatizer = WordNetLemmatizer()

def get_lem(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [25]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [26]:
get_lem(tweet)

'RT @RawStory: Researchers say we have three year to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

### Tokenising 

Write a function that tokenises a single tweet into a list of tokens.

**Function input:**
- A single string object (tweet) 

**Function output:**
- A list of tokens (objects).

In [27]:
tokeniser = TreebankWordTokenizer()

**Original object:**

In [28]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [29]:
tokeniser.tokenize(tweet)

['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']

**Expected output:**
```python
get_tokens(tweet)

>> ['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']
```

Besides the preprocessing methods we've already discussed, can you think of how our data can be improved at this stage?

### Vectorising 

Use the code from the webinar to define your vecotiser. For more details on this function have a look at the `sklearn`
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">package documentation on the function</a>. Recall that this function, unlike the others before takes an _**entire column**_ as an argument.

### Putting all the functions together 

Test your cleaning functions on a single tweet. Perform the cleaning procedure on this tweet from start to finish. Make sure the output looks as you would expect. Store the cleaned tweet in `tweet_final`. Don't worry about vectorising for now.

**_Include any other cleaning procedures you add after your base model_**

**Original object:**

In [30]:
tt = TfidfVectorizer(preprocessor=list,tokenizer=list,ngram_range=(1,2),min_df=2,strip_accents='ascii',smooth_idf=False)

In [31]:
tt.fit(train['message'])
vect = tt.transform(train['message'])

C:\Users\lizet\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [33]:
# clean tweet from start to finish
tweet1 = remove_urls(tweet)
tweet2 = Correct_spelling(tweet1)
tweet3 = to_lower(tweet2)
tweet4 = remove_handle(tweet3)
tweet5 = remove_punc(tweet4)
tweet6 = get_lem(tweet5)
tweet7 = tokeniser.tokenize(tweet6)


# cleaned tweet
tweet_final = tweet7
tweet_final

['rt',
 'researcher',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 '…']

**Expected output:**
```python
tweet_final
>>['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']
```

### Preprocessing and cleaning entire column of tweets

Now that we know how to clean a single tweet, we can use `apply()` to clean the entire column in the dataframe. This is what the column looks like to begin with.

In [34]:
train['message'].head()

0    PolySciMajor EPA chief doesn't think carbon di...
1    It's not like we lack evidence of anthropogeni...
2    RT @RawStory: Researchers say we have three ye...
3    #TodayinMaker# WIRED : 2016 was a pivotal year...
4    RT @SoyNovioDeTodas: It's 2016, and a racist, ...
Name: message, dtype: object

Write code that applies your cleaning functions to the entire `message` column of the dataframe using `.apply()`. Do not vectorise just yet. Store this in a new column, `train['message_clean']`. 

In [35]:
# applying cleaning functions to entire column of dataframe (NO VECTORISING)

# YOUR CODE HERE - use apply with all your cleaning functions 

train['message_clean'] = train['message'].apply(remove_urls).apply(Correct_spelling).apply(remove_punc).apply(to_lower).apply(get_lem).apply(tokeniser.tokenize).apply(stop_words)


train['message_clean'].head()

0    [polyscimajor, epa, chief, doesnt, think, carb...
1    [like, lack, evidence, anthropogenic, global, ...
2    [rt, rawstory, researcher, say, three, year, a...
3    [todayinmaker, wired, 2016, wa, pivotal, year,...
4    [rt, soynoviodetodas, 2016, racist, sexist, cl...
Name: message_clean, dtype: object

**Expected output:**

```python
train['message_clean'].head()

>>  0    [polyscimajor, epa, chief, does, n't, think, c...
    1    [it, not, like, we, lack, evid, of, anthropoge...
    2    [rt, @, rawstory, :, research, say, we, have, ...
    3    [#, todayinmaker, #, wire, :, 2016, was, a, pi...
    4    [rt, @, soynoviodetodas, :, it, 2016, ,, and, ...
Name: message_clean, dtype: object
```

### Vectorising 

Vectorise your cleaned data using the vectoriser you defined earlier. Don't forget to fit the vectoriser to the data you just cleaned. Store your vectorised data in `train_vec`.

In [36]:
# vectorise your cleaned data
train_vec = tt.transform(train['message_clean'])
train_vec.shape
# YOUR CODE HERE 

(15819, 6181)

In [37]:
train_vec.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Fitting a simple model on training data and evaluate its performance

Fit a model on your cleaned data. Use `train_vec` as your features. Note that you need to convert `train_vec` to an array. 

You are familiar with this process, so there is less guidance here. You may also refer to train notebooks and the webinar pdf as guidance.

Your basic model should be a logistic regression, however if you try different models you will also perform the following procedure.

1. Split the training data into features and labels.
2. Split the training data further into training and validation data.
3. Fit the model on the training subset. 
4. Predict on the validation subset.
5. Calculate the performance metrics on the validation predictions.
6. Select a model based on validation performance (when you have more than one model).
8. Clean the test data.
9. Predict on the cleaned test data.
7. Write a csv that matches `sample_submission.csv`.
8. Submit to Kaggle.

In [38]:
# YOUR CODE HERE 
y = train['sentiment'].apply(str)

y.head()


0    1
1    1
2    2
3    1
4    1
Name: sentiment, dtype: object

In [39]:
X = train_vec.toarray()
X.shape

(15819, 6181)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)
y_test

9129     0
7175     1
11914    1
6870     1
6700     1
        ..
3789     1
12153    1
4993     1
3934     0
2691     1
Name: sentiment, Length: 3164, dtype: object

In [41]:
lr = LogisticRegression(solver = 'liblinear', random_state = 2022)
model_lr = lr.fit(X_train, y_train)
model_lr.coef_

array([[ 0.        ,  0.        ,  0.        , ..., -0.14362186,
         0.        , -0.14388936],
       [ 0.        ,  0.        ,  0.        , ...,  0.52529192,
         0.        ,  0.52579148],
       [ 0.        ,  0.        ,  0.        , ..., -0.05629988,
         0.        , -0.05707575],
       [ 0.        ,  0.        ,  0.        , ..., -0.33347769,
         0.        , -0.33311646]])

In [42]:
lr.intercept_[0]

-2.391664073639333

In [43]:
y_pred_test = lr.predict(X_test)
target_names = ['-1','0','1','2']
y_pred_test


array(['1', '1', '1', ..., '1', '1', '1'], dtype=object)

In [44]:
confusion_matrix(y_test,y_pred_test)

array([[   0,    1,  260,    0],
       [   0,    1,  478,    0],
       [   0,    2, 1648,    7],
       [   0,    0,  763,    4]], dtype=int64)

In [45]:
pd.DataFrame(data=confusion_matrix(y_test, y_pred_test), index=target_names, columns=target_names)

,-1,0,1,2
-1,0,1,260,0
0,0,1,478,0
1,0,2,1648,7
2,0,0,763,4


In [46]:
lm_vt = LogisticRegression(solver = 'liblinear', random_state = 2022)
selector = VarianceThreshold(threshold=0.02)

# Transform (i.e.: run selection on) the training data
X_vt = selector.fit_transform(X_train)

In [47]:
X_vt1 = lm_vt.fit(X_vt, y_train)
X_vt1.coef_

array([[-0.28811532, -0.37684746],
       [-0.15210453, -0.82941877],
       [-0.14754589, -0.23073603],
       [ 0.40337301,  0.80423294]])

In [48]:
X_vt1.intercept_[0]

-2.379547009982013

In [49]:
X_test_vt = selector.transform(X_test)

In [50]:
y_pred_vt = X_vt1.predict(X_test_vt)
target_names = ['-1','0','1','2']
y_pred_vt

array(['1', '1', '1', ..., '1', '1', '1'], dtype=object)

In [51]:
from sklearn.metrics import classification_report

In [52]:
f1_score(y_test, y_pred_vt,average = 'macro')

0.17185231279817464

In [53]:
lm_kbest = LogisticRegression(solver = 'liblinear', random_state = 2022)

In [54]:
from sklearn import feature_selection
from sklearn.feature_selection import f_classif

# Set up selector, choosing score function and number of features to retain
selector_kbest = feature_selection.SelectKBest(score_func=f_classif, k = 20)

# Transform (i.e.: run selection on) the training data
X_train_kbest = selector_kbest.fit_transform(X_train, y_train)

C:\Users\lizet\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   0    1    2 ... 6176 6177 6179] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [55]:
lm_kbest.fit(X_train_kbest, y_train)

LogisticRegression(random_state=2022, solver='liblinear')

In [56]:
X_test_kbest = selector_kbest.transform(X_test)

# Generate predictions from var thresh model
pred_lm_kbest = lm_kbest.predict(X_test_kbest)

In [57]:
print('Classification Report')
print(classification_report(y_test, y_pred_test))

print('Classification Report')
print(classification_report(y_test, y_pred_vt))

print('Classification Report')
print(classification_report(y_test, pred_lm_kbest))


Classification Report
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       261
           0       0.25      0.00      0.00       479
           1       0.52      0.99      0.69      1657
           2       0.36      0.01      0.01       767

    accuracy                           0.52      3164
   macro avg       0.28      0.25      0.18      3164
weighted avg       0.40      0.52      0.36      3164

Classification Report


C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metr

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       261
           0       0.00      0.00      0.00       479
           1       0.52      1.00      0.69      1657
           2       0.00      0.00      0.00       767

    accuracy                           0.52      3164
   macro avg       0.13      0.25      0.17      3164
weighted avg       0.27      0.52      0.36      3164

Classification Report
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       261
           0       0.25      0.00      0.00       479
           1       0.52      0.99      0.69      1657
           2       0.36      0.01      0.01       767

    accuracy                           0.52      3164
   macro avg       0.28      0.25      0.18      3164
weighted avg       0.40      0.52      0.36      3164



C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\lizet\anaconda3\lib\site-packages\sklearn\metr

## Suggestions on how to improve the model - Extras 

Here are some ideas on how you can improve your model. Try some of them out once you have submitted your base model to Kaggle. Only use the methods you find improve your model performance.

 - Additional preprocessing 
     - Use some of the methods in the notebooks to balance the data
     - Replace links with the text `Link` to show the machine that there is a link present without the extraneous detail of the link text
     - Remove punctuation
     - Correct spelling
     - Bag of words vs. N-grams - which is better?
     - Try `TweetTokenizer()` - how does this differ from the tokenisers you are familiar with?
 - Consider which other models could do a better job (refer to course resources)
 - Consider some further feature engineering:
     - What other attributes of the tweets may be useful?
     - Should we perform feature selection given their size and sparseness of the vectorised tweets?
 - If you've mastered the above, try implementing hyperparameter tuning methods.
 - Compare all the models you try and remember to submit **the best one** based on test performance (f1-score using macro averaging). 
 
You may use the space below to try out these "Extras".

In [58]:
# Extras
# YOUR CODE HERE 




## How/why is the model bad?

Do you have any hypotheses as to how you can improve the model further? Use this space to investigate with mindful EDA if necessary.

In [59]:
# EDA
# YOUR CODE HERE 





Be sure to check whether each additional technique you apply actually improves your model performance.

## Submit your best model predictions 

Don't forget to submit your the predictions on the best model to Kaggle.

In [60]:
# write csv for best predictions




In [61]:
test['message_clean'] = test['message'].apply(remove_urls).apply(Correct_spelling).apply(to_lower).apply(get_lem).apply(tokeniser.tokenize)


test['message_clean'].head()

0    [europe, will, now, be, looking, to, china, to...
1    [combine, this, with, the, polling, of, staffe...
2    [the, scary, ,, unimpeachable, evidence, that,...
3    [@, karoli, @, morgfair, @, osborneink, @, dai...
4    [rt, @, fakewillmoore, :, 'female, orgasm, cau...
Name: message_clean, dtype: object

In [62]:
test_vec = tt.transform(test['message_clean'])
test_vec.shape

(10546, 6181)

In [63]:
X1 = test_vec.toarray()
X1.shape

(10546, 6181)

In [64]:
y_pred = lr.predict(X1)

In [65]:
tweet_id = test['tweetid']

In [66]:
submission = pd.DataFrame({'tweetid': tweet_id, 'sentiment':y_pred})

In [67]:
submission.head()

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,1


In [68]:
submission.to_csv('Lizette_Da_Ascencao.csv', index=False)